In [43]:
from playwright.async_api import async_playwright
from llm import GeminiUsage, call_llm, call_openai
from termcolor import cprint

# p = await async_playwright().start()
# browser = await p.chromium.launch(headless=False)
# page = await browser.new_page(no_viewport=True)
# await page.goto("https://www.target.com/")
# await p.stop()

def llm(prompt):
    cprint(prompt, "black")
    cprint(call_llm(prompt), "green")

# Our Goal: Build an autonomous agent that can browse the web

### Task: "Order a large pepperoni pizza from Domino's for me"


In [44]:
task = "Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107"

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

What is the most logical next action to take?
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

What is the most logical next action to take?

Open the Domino's website or app.


# Step 1: Give the agent eyes!
To order a pizza, the agent needs to know what the webpage looks like. So let's pass the HTML to the LLM so it can "see" the webpage.

- This is RAG (retrieval augmented generation)!

In [51]:
currently_open_url = "https://www.dominos.com/"
fake_dominos_html = """<html>
    <body>
        <header>
            <h1>Welcome to Domino's Pizza</h1>
            <p>Order online for delivery or pickup</p>
            <a href="#" id="order-button">Order Now</a>
        </header>

        <main>
            <h2>Our Specialties</h2>
            <p>Discover our delicious range of pizzas, sides, and desserts.</p>
        </main>
    </body>
</html>"""

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide what is the most logical next action to take.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide what is the most logical next action to take.

The webpage https://www.dominos.com/ is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

<html>
    <body>
        <header>
            <h1>Welcome to Domino's Pizza</h1>
            <p>Order online for delivery or pickup</p>
            <a href="#" id="order-button">Order Now</a>
        </header>

        <main>
            <h2>Our Specialties</h2>
            <p>Discover our delicious range of pizzas, sides, and desserts.</p>
        </main>
    <

# Step 2: Tell the agent what actions it can perform
The agent should be able to click on buttons, go to a url, and fill out form fields

In [48]:
actions_agent_can_perform = """\
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID
fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.
choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.
go_to_url(url: 'str'): Open a webpage by URL"""

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You can perform the following actions to interact with the browser:
{actions_agent_can_perform}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide the single most logical next action to take to help advance in achieving the task.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You can perform the following actions to interact with the browser:
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID
fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.
choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.
go_to_url(url: 'str'): Open a webpage by URL

You must carefully read over the current webpage's HTML, and ba

# Step 3: Tell the agent to return a JSON
We want the agent to output the response in a structured format so we can parse the action and execute it outside of the LLM!

- Alternatively you can use the ChatGPT function calling API / structured output API

In [52]:
json_structure = """\
{
    "action": "Call the action in the following format: action_name(param_name='argument')"
}
"""

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You can perform the following actions to interact with the browser:
{actions_agent_can_perform}

Your response must be in JSON format with a field named "action":
{{
    "action": "Call the action in the following format: action_name(param_name='argument')"
}}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide the single most logical next action to take to help advance in achieving the task.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You can perform the following actions to interact with the browser:
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID
fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.
choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.
go_to_url(url: 'str'): Open a webpage by URL

Your response must be in JSON format with a field named "action

# Step 4: Make the agent think before taking an action

LLMs are prone to hallucination. If you ask the agent to explicitly describe what it observes in the HTML and to **reason** out loud, it is much more likely to arrive at a logical decision. Without this observation/reasoning step, the agent might hallucinate and:

- Make up HTML IDs that don't exist
- Choose an incorrect action that doesn't advance the task

In [57]:
better_json_structure = """\
{
    "action": "Call the action in the following format: action_name(param_name='argument')"
}
"""

prompt = f"""\
You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
{task}

You can perform the following actions to interact with the browser:
{actions_agent_can_perform}

Your response must be in JSON format with fields named "observations", "reasoning", and "action":
{{
    "observations": "Carefully read the HTML of the current webpage. Based on the HTML, explain the purpose of the webpage and identify the important HTML elements. Pay close attention to the state of informational elements such as the cart, input validation errors, sign in status, etc.",
    "reasoning": "Think critically about what action you can perform on the HTML to help advance in the user task. Describe why you think this is the best action to perform given the contents of the current page.",
    "action": "Call the action in the following format: action_name(param_name='argument')"
}}

You must carefully read over the current webpage's HTML, and based on the current state of the webpage, decide the single most logical next action to take to help advance in achieving the task.

The webpage {currently_open_url} is open. Carefully analyze the HTML, and based on the HTML contents, determine the next action to take to help the user get closer to achieving their task. The HTML of the current webpage is:

{fake_dominos_html}
"""

llm(prompt)

You are a helpful assistant who can interact with a browser on behalf of the user. Your purpose is to help a user complete the following task:
Order me a large pepperoni pizza from Domino's for delivery to 275 Brannan St, San Francisco, CA 94107

You can perform the following actions to interact with the browser:
click_html_element(id: 'str'): Click an HTML <a> tag or <button> identified by its ID
fill_text_in_input(id: 'str', text: 'str'): Type text into an input or textarea identified by its ID. Important: This function can ONLY be called with an ID that belongs directly to an <input> or <textarea> tag.
choose_dropdown_values(id: 'str', values: 'List[str]'): Select value(s) for a <select> tag identified by its ID. The `values` list must contain at least one string option to select. Important: This function can ONLY be called with an ID that belongs directly to a <select> tag.
go_to_url(url: 'str'): Open a webpage by URL

Your response must be in JSON format with fields named "observa